In [2]:
import os
import json
import socket
import requests

from transformers import AutoConfig
from transformers import AutoTokenizer

from sentence_transformers import util
from sentence_transformers import SentenceTransformer

In [3]:
with open('config.json', 'r') as file:
    config = json.load(file)
print("LOADED\t\t\t: config")

LOADED			: config


## Test question_checker

In [16]:
def question_checker(question, refq, treshold=0.5, model=None):
    if model is not None:
        emb_refq = model.encode(refq)
        emb_actq = model.encode([question])

        cosine_similarities = util.cos_sim(emb_refq, emb_actq)
        # print([[sim > treshold, sim] for sim in cosine_similarities[0]])
        return any(sim > treshold for sim in cosine_similarities)
    else:
        return False

In [5]:
model_st = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [17]:
question = "What is the bread?"
positive_question_1 = "What is the bread?"
positive_question_2 = "What is the soft bread?"
neagtive_question_1 = "What is the formula one car?"
negative_question_2 = "How old are you?"
    
assert question_checker(question, [positive_question_1], 0.99, model=model_st) == True
assert question_checker(question, [positive_question_2], 0.99, model=model_st) == False
assert question_checker(question, [positive_question_2], 0.5, model=model_st) == True
assert question_checker(question, [neagtive_question_1], 0.5, model=model_st) == False
assert question_checker(question, [negative_question_2], 0.1, model=model_st) == False

: 

## Test cleaning_stream

In [6]:
def cleaning_stream(batch):
    if len(batch['text_output']) == 0:
        return False
    elif batch['text_output'] == "\n\n":
        return False
    elif "<|start_header_id|>" in batch['text_output']:
        return False
    elif "assistant" in batch['text_output']:
        return False
    elif "<|end_header_id|>" in batch['text_output']:
        return False
    else:
        return True

In [7]:
batch = {'text_output': ""}
assert cleaning_stream(batch) == False

# 2. Csak új sorokból áll
batch = {'text_output': "\n\n"}
assert cleaning_stream(batch) == False

# 3. "<|start_header_id|>" szöveget tartalmaz
batch = {'text_output': "This is some text with <|start_header_id|> in it"}
assert cleaning_stream(batch) == False

# 4. "assistant" szöveget tartalmaz
batch = {'text_output': "This is some text with assistant in it"}
assert cleaning_stream(batch) == False

# 5. "<|end_header_id|>" szöveget tartalmaz
batch = {'text_output': "This is some text with <|end_header_id|> in it"}
assert cleaning_stream(batch) == False

# 6. Minden feltételnek megfelelő szöveg (nincs tiltott karakter vagy string)
batch = {'text_output': "This is a valid text output"}
assert cleaning_stream(batch) == True

## Test reduce_message

In [8]:
tokenizer = AutoTokenizer.from_pretrained(config["tokenizer"])
print("TOKENIZER\t\t:", config["tokenizer"])

TOKENIZER		: meta-llama/Llama-3.1-8b-instruct


In [9]:
autoconfig = AutoConfig.from_pretrained(config["tokenizer"])
limit_pe = autoconfig.max_position_embeddings - config["guard"]["reduce_max_position_embeddings"]
print("MAX TOKEN INPUT SIZE\t:", autoconfig.max_position_embeddings)

MAX TOKEN INPUT SIZE	: 131072


In [10]:
def reduce_message(chat, max_len, num_limit, tokenizer):
    message = tokenizer.apply_chat_template(chat, tokenize=False)
    len_chat = len(tokenizer.encode(message))      
    while len_chat > max_len and len(chat) > num_limit:
        chat = chat[0:1] + chat[2:]

        message = tokenizer.apply_chat_template(chat, tokenize=False)
        len_chat = len(tokenizer.encode(message))
        # print(len_chat, chat)

    if len_chat < max_len:
        return message
    else:
        return ""

In [11]:
chat = [
    {"role": "system", "content": "Some content."},
    {"role": "assistant", "content": "Some content."},
    {"role": "user", "content": "Some content."},
    {"role": "assistant", "content": "Some content."},
    {"role": "user", "content": "Some content."}]

message = tokenizer.apply_chat_template(chat, tokenize=False)
print(len(tokenizer.encode(message)))
print(message)

# print()

assert len(tokenizer.encode(reduce_message(chat, 60, 3, tokenizer))) <= 60
assert len(tokenizer.encode(reduce_message(chat, 50, 3, tokenizer))) <= 50
assert len(tokenizer.encode(reduce_message(chat, 40, 3, tokenizer))) == 1

message = reduce_message(chat, 50, 3, tokenizer)
print(len(tokenizer.encode(message)))
print(message)

62
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Some content.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Some content.<|eot_id|><|start_header_id|>user<|end_header_id|>

Some content.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Some content.<|eot_id|><|start_header_id|>user<|end_header_id|>

Some content.<|eot_id|>
46
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Some content.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Some content.<|eot_id|><|start_header_id|>user<|end_header_id|>

Some content.<|eot_id|>


## Test Server

In [12]:
url = f"{config['triton']['host']}:{config['triton']['port']}/v2/{config['triton']['model']}/{config['triton']['generation']}"
print("URL\t\t\t:", url)

URL			: http://172.22.214.120:28800/v2/models/llama-3.1-8b-instruct/generate_stream


In [13]:
question = "Mennyi volt az átlagos páratartalom 2024.09.16-án 10 órakkor."
question

'Mennyi volt az átlagos páratartalom 2024.09.16-án 10 órakkor.'

In [14]:
if question_checker(question=question, 
                    refq=config["filters"]["reference questions"], 
                    treshold=config["filters"]["treshold"], 
                    model=model_st):
    print("Ok")
else:
    print(config["guard"]["no information"])

Ok


In [15]:
with open(f"{config['sources']}/total.json", 'r') as file:
    data = json.load(file)
print("LOADED\t\t\t: data")

LOADED			: data


In [16]:
reinforce = " Csak a kérdésre válaszolj! Ne magyarázd a válaszod és ne sorolj fel részadatokat de teljes mondatban válaszolj!"
chat = config["chat history"].copy()
chat[-1]["content"] = f"Páratartalom adatai (JSON):{str(data)}\n\n"
chat.append({"role": "user", "content": question + reinforce})

message = tokenizer.apply_chat_template(chat, tokenize=False)

len(tokenizer.encode(message))

10704

In [19]:
message

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nPáratartalom adatai (JSON):[{'_óra_': 1, '_átlag_': '58.525%', '_év_': '2024', '_hónap_': '09', '_nap_': '10', '_dátum_': '2024.09.10 1 óra/h'}, {'_óra_': 2, '_átlag_': '45.433%', '_év_': '2024', '_hónap_': '09', '_nap_': '10', '_dátum_': '2024.09.10 2 óra/h'}, {'_óra_': 3, '_átlag_': '43.525%', '_év_': '2024', '_hónap_': '09', '_nap_': '10', '_dátum_': '2024.09.10 3 óra/h'}, {'_óra_': 4, '_átlag_': '47.317%', '_év_': '2024', '_hónap_': '09', '_nap_': '10', '_dátum_': '2024.09.10 4 óra/h'}, {'_óra_': 5, '_átlag_': '47.22%', '_év_': '2024', '_hónap_': '09', '_nap_': '10', '_dátum_': '2024.09.10 5 óra/h'}, {'_óra_': 6, '_átlag_': '51.567%', '_év_': '2024', '_hónap_': '09', '_nap_': '10', '_dátum_': '2024.09.10 6 óra/h'}, {'_óra_': 7, '_átlag_': '42.883%', '_év_': '2024', '_hónap_': '09', '_nap_': '10', '_dátum_': '2024.09.10 7 óra/h'}, {'_óra_': 8, '_átlag_': 

: 

In [17]:
if autoconfig.max_position_embeddings > len(tokenizer.encode(message)):
    payload = {
        "text_input": message,
        "max_tokens": config['triton']['max_tokens'],
        "temperature": config['triton']['temperature'],
        "stream": ("stream" in config['triton']['generation'])
    }

    # Kérelem küldése és stream feldolgozása
    with requests.post(url, json=payload, stream=True) as response:
        # Ellenőrizzük, hogy sikeres volt-e a válasz
        if response.status_code == 200:
            # Streameljük a válasz sorokat
            for line in response.iter_lines():
                if line:
                    # JSON-adat soronkénti feldolgozása
                    batch = json.loads(line.decode('utf-8').replace("data: ", ""))
                    if cleaning_stream(batch):
                        print(batch['text_output'], end='', flush=True)
                    # print(data['text_output'], end='\n', flush=True)
        else:
            print(f"Hiba történt: {response.status_code}")
else:
    print("Need cutting")

Az átlagos páratartalom 2024.09.16-án 10 órakor 53,483% volt.

In [39]:
# import requests
# import json

# # A POST URL-je a model híváshoz
# url = "http://172.22.214.120:28800/v2/models/llama-3.2-1b-instruct/generate"
# # url = "http://172.22.214.120:28800/v2/models/llama-3.1-8b-instruct/generate"

# # A kéréshez tartozó payload
# payload = {
#     "text_input": "machine learning is",
#     "max_tokens": 128,
#     "temperature": 0.95
# }

# # Kérelem küldése és a válasz feldolgozása
# response = requests.post(url, json=payload)

# # Ellenőrizzük, hogy sikeres volt-e a válasz
# if response.status_code == 200:
#     # Teljes JSON válasz feldolgozása
#     data = response.json()
#     print(data['text_output'])
# else:
#     print(f"Hiba történt: {response.status_code}")


## JSON convert

In [ ]:
# system_prompt = f"""Egy segítőkész és tisztelettudó szenzoros rendszert támogató asszisztens vagy aki a következő instrukciók alapján viselkedik:
# - Mindig formálist stílusban válaszolsz a felhasználók kérdéseire.
# - A kérdések megválaszolásához ne írj python vagy más programozási nyelvben kódot.
# - Csak akkor válaszolsz a felhasználónak, ha a következő témakörökkel kapcsolatban kérdez:
# 	- A szenzoros adatokból kiolvasható átlagos páratartalom.
# 	- A szenzoros adatokat megjelenítő dashboard felhasználói kézikönyve.
# - Ha felhasználó nem a megadott témakörökkel kapcsolatban teszi fel a kérdését akkor udvariasan utasítsd vissza a válaszadást. Jelezd, hogy csak a következőkkel kapcsolatban válaszolhatsz a kérdésekre:
# 	- A szenzoros adatokból kiolvasható átlagos páratartalom.
# 	- A szenzoros adatokat megjelenítő dashboard felhasználói kézikönyve.
# - A kérdések megválaszolásánál mindig figyelembe veszed a rendelkezésedre álló páratartalom adatokat, amely a szenzorok mérési eredményeit tartalmazzák. _óra_, _átlag_, _év_, _hónap_, _nap_ jellemzőkkel. Az egyes jellemzők pontos leírása a következő:
# 	- _átlag _ : Megadja az adott mérési ponthoz tartozó átlagos páratartalom értékét %-ban.
# 	- _év_ : Megadja hogy az _átlag_ jellemzőhöz tartozó páratartalom érték melyik évben került megmérésre, rögzítésre.
# 	- _hónap_ : Megadja hogy az _átlag_ jellemzőhöz tartozó páratartalom érték az adott év melyik hónapjában került megmérésre, rögzítésre.
# 	- _nap_ : Megadja hogy az _átlag_ jellemzőhöz tartozó páratartalom érték az adott év adott hónapjának melyik napján került megmérésre, rögzítésre.
# 	- _óra_ : Megadja hogy az _átlag_ jellemzőhöz tartozó páratartalom érték az adott napon belül mikor, hány órakor lett megmérve.
# 	- _dátum_ : Megadja hogy az _átlag_ jellemzőhöz tartozó páratartalom mikor került, rögzítésre év.hónap.nap óra formátumban.
# - Kérdésekre mindig rövid és tömör választ adsz.
# - Ha felhasználó elköszön tőled akkor te is udvariasan elköszönsz."""

# print(system_prompt.replace("\n", "\\n").replace("\t", "\\t"))

In [7]:
system_prompt = f"""Egy segítőkész és tisztelettudó szenzoros rendszert támogató asszisztens vagy aki a következő instrukciók alapján viselkedik:
- Mindig formális stílusban válaszolsz a felhasználók kérdéseire.
- A kérdések megválaszolásához ne írj python vagy más programozási nyelvben kódot.
- Csak akkor válaszolsz a felhasználónak, ha a következő témakörökkel kapcsolatban kérdez:
	- A szenzoros adatokból kiolvasható átlagos páratartalom.
	- A szenzoros adatokat megjelenítő dashboard felhasználói kézikönyve.
- Ha felhasználó nem a megadott témakörökkel kapcsolatban teszi fel a kérdését akkor udvariasan utasítsd vissza a válaszadást. Jelezd, hogy csak a következőkkel kapcsolatban válaszolhatsz a kérdésekre:
	- A szenzoros adatokból kiolvasható átlagos páratartalom.
	- A szenzoros adatokat megjelenítő dashboard felhasználói kézikönyve.
- A kérdések megválaszolásánál mindig figyelembe veszed a rendelkezésedre álló páratartalom adatokat, amely a szenzorok mérési eredményeit tartalmazzák.
- Kérdésekre mindig rövid és tömör választ adsz.
- Ha felhasználó elköszön tőled akkor te is udvariasan elköszönsz."""

print(system_prompt.replace("\n", "\\n").replace("\t", "\\t"))

Egy segítőkész és tisztelettudó szenzoros rendszert támogató asszisztens vagy aki a következő instrukciók alapján viselkedik:\n- Mindig formális stílusban válaszolsz a felhasználók kérdéseire.\n- A kérdések megválaszolásához ne írj python vagy más programozási nyelvben kódot.\n- Csak akkor válaszolsz a felhasználónak, ha a következő témakörökkel kapcsolatban kérdez:\n\t- A szenzoros adatokból kiolvasható átlagos páratartalom.\n\t- A szenzoros adatokat megjelenítő dashboard felhasználói kézikönyve.\n- Ha felhasználó nem a megadott témakörökkel kapcsolatban teszi fel a kérdését akkor udvariasan utasítsd vissza a válaszadást. Jelezd, hogy csak a következőkkel kapcsolatban válaszolhatsz a kérdésekre:\n\t- A szenzoros adatokból kiolvasható átlagos páratartalom.\n\t- A szenzoros adatokat megjelenítő dashboard felhasználói kézikönyve.\n- A kérdések megválaszolásánál mindig figyelembe veszed a rendelkezésedre álló páratartalom adatokat, amely a szenzorok mérési eredményeit tartalmazzák.\n- Kér